In [2]:
import rasterio as rio 
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd 
import matplotlib.patches as mpatches 
from matplotlib.colors import ListedColormap
from rasterio.enums import Resampling 
from PIL import ImageColor
from skimage.exposure import rescale_intensity
import json
import os

In [ ]:
lc_dir = "lc.json"
lc_image_dir = 'https://storage.googleapis.com/gee-ramiqcom-bucket/youtube/lc_python/LC_Image_v1.tif'
palsar_dir = "https://storage.googleapis.com/gee-ramiqcom-bucket/youtube/lc_python/Palsar_Kerinci_2023_v1.tif"
landsat_dir = "https://storage.googleapis.com/gee-ramiqcom-bucket/youtube/lc_python/Landsat_Kerinci_2023_v1.tif"

In [ ]:
lc = json.load(open(lc_dir))
lc_df = pd.DataFrame(lc)
lc_df["values_normalize"] = lc_df.index +1 
lc_df["palette"] = "#" + lc_df["palette"]

values = lc_df["values"].to_list()
values_norm = lc_df["values_normalize"].to_list()
palette = lc_df["palette"].to_list()
labels = lc_df["label"].to_list()

dict_values = {}
dict_label = {}
dict_palette = {}
dict_palette_hex = {}

for x in range(0,len(values)):
    dict_values[values[x]] = values_norm[x]
    dict_label[values_norm[x]] = labels[x]
    dict_palette[values_norm[x]] = ImageColor.getrgb(palette[x])
    dict_palette_hex[values_norm[x]] = palette[x]
    
## colormap from values and palette
cmap = ListedColormap(palette) ## defines a colormap with our chosen class colors , example ; “Whenever you see value 1, color it #006400 ; value 2, color it #228B22; …”

patches = [ ## creates colored squares for the legend.
    mpatches.Patch(color=palette[i], label=labels[i]) for i in range(len(values))
]    

legend = {
    "handles":patches,
    "bbox_to_anchor":(1.05,1),
    "loc":2,
    "borderaxespad":0.0,
}
lc_df

In [3]:
lc_values = os.listdir("C://Users//DHIA//Desktop//Satellite Imagery Land Cover Classification//data//images")
lc_values = list(map(lambda x: int(x.split('_')[0]),lc_values))
lc_values = np.unique(lc_values)
lc_values

array([ 1,  2,  5,  6,  7,  8,  9, 11, 14, 15, 16, 17, 20, 22])

In [4]:
## Data Augmentation Function

def augent(array,out_list):
    flip=np.flip(array,1)
    for arr in [array,flip]:
        for rot in [1,2,3]:
            out_list.append(np.rot90(arr,rot))

In [ ]:
# image per land cover

image_per_lc = 20 
index_train = int(8/10*image_per_lc)
plt.figure(figsize=(4*image_per_lc,4*len(lc_values)))
